<a href="https://colab.research.google.com/github/thaispatricio9/compras_analise_BI/blob/main/02_json_to_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json

## Transformar o JSON em Dataframe

In [44]:
url = 'https://raw.githubusercontent.com/thaispatricio9/compras_analise_BI/main/data.json'

In [45]:
df = pd.read_json(url)

In [46]:
df

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ItemList
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,"[{'ProductName': 'Rice', 'Value': 35.55, 'Quan..."
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,"[{'ProductName': 'Tomate', 'Value': 12.25, 'Qu..."
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,"[{'ProductName': 'Beer', 'Value': 9.0, 'Quanti..."


Verifica-se que a coluna ItemList contêm uma lista com os produtos, valor e quantidade de cada nota fiscal. Seguindo o modelo relacional irei criar a tabela fato contendo a data de criação, emissão, desconto e nota_fiscal e a tabela dimensão com o detalhamento do produto de cada nota fiscal

In [47]:
df_list = (pd.concat({k: pd.DataFrame(v) for k, v in df.pop('ItemList').items()}).reset_index(level=1, drop=True))

dataset = df.join(df_list, rsuffix='_').reset_index(drop=True)

In [48]:
dataset

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ProductName,Value,Quantity
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Rice,35.55,2
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Flour,11.55,5
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Bean,27.15,7
3,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,Tomate,12.25,10
4,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,Pasta,7.55,5
5,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,Beer,9.00,6
6,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,French fries,10.99,2
7,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,Ice cream,27.15,1


In [50]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CreateDate    8 non-null      object 
 1   EmissionDate  8 non-null      object 
 2   Discount      8 non-null      int64  
 3   NFeNumber     8 non-null      int64  
 4   NFeID         8 non-null      int64  
 5   ProductName   8 non-null      object 
 6   Value         8 non-null      float64
 7   Quantity      8 non-null      int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 640.0+ bytes


## Modelo Relacional

Tabela Dimensão da Nota Fiscal contendo os produtos, valores e quantidade referentes a cada NFeID

In [51]:
dim_nota_fiscal  = df[['NFeID']].join(df_list, rsuffix='_').reset_index(drop=True)

In [52]:
dim_nota_fiscal

,NFeID,ProductName,Value,Quantity
0,1,Rice,35.55,2
1,1,Flour,11.55,5
2,1,Bean,27.15,7
3,2,Tomate,12.25,10
4,2,Pasta,7.55,5
5,3,Beer,9.00,6
6,3,French fries,10.99,2
7,3,Ice cream,27.15,1


Tabela Fato contendo os pedidos por data

In [56]:
# Como fiz um drop do ItemList na união das tabelas, a tabela fato é a mesma que o dataframe df
fato = df

In [57]:
fato

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3


## Salvar arquivos

In [58]:
fato.to_csv('fato.csv')
dim_nota_fiscal.to_csv('dim_nfe.csv')